### In this notebook, we demonstrate a common OOM error when reading parquet files using dask-cudf. The error is caused by too large row groups in the parquet file. To workaround this issue, we simply rewrite the parquet files with smaller row groups.

In [1]:
import dask_cudf
import numpy
import pandas as pd
import pyarrow.dataset as ds

### Frist, we generate a random dataframe

In [2]:
df = pd.DataFrame()
for i in range(100):
    df[f'fea{i}'] = numpy.random.rand(25_000_000)

### Note that the dataframe is 18 GB in memory while a single GPU has 32 GB memory. There should be no problem for gpu to read this parquet file.

In [3]:
print(f'dataframe size: {df.memory_usage().sum()/2**30:.2f} GB')

dataframe size: 18.63 GB


### We create this OOM error by forcing only one row group when writing to parquet

In [4]:
df.to_parquet('x.parquet',row_group_size=df.shape[0], engine='pyarrow')

In [5]:
print('number of row groups:')
list(ds.dataset('x.parquet', format="parquet").get_fragments())[0].num_row_groups

number of row groups:


1

### OOM occurs as expected

In [6]:
ddf = dask_cudf.read_parquet('x.parquet')
ddf.shape[0].compute()

MemoryError: Parquet data was larger than the available GPU memory!

See the notes on split_row_groups in the read_parquet documentation.

Original Error: std::bad_alloc: out_of_memory: CUDA error at: /raid/data/mambaforge/envs/rapids-23.06/include/rmm/mr/device/cuda_memory_resource.hpp

### To fix this error, we simply rewrite the parquet file with smaller row group size

In [7]:
df.to_parquet('x.parquet', row_group_size=1_000_000)

In [8]:
print('number of row groups:')
list(ds.dataset('x.parquet', format="parquet").get_fragments())[0].num_row_groups

number of row groups:


25

In [9]:
ddf = dask_cudf.read_parquet('x.parquet')
ddf.shape[0].compute()

25000000